In [1]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import numpy as np

# Cargar los datos como en tu código original
# Asumiendo que ya tienes X_train_scaled, X_val_scaled, y_train_encoded, y_val_encoded preparados

def create_model(trial):
    # Optimización de hiperparámetros
    # Definir el número de neuronas por capa
    num_neurons_1 = trial.suggest_int('num_neurons_1', 32, 128)
    num_neurons_2 = trial.suggest_int('num_neurons_2', 16, 64)
    
    # Regularización L2
    l2_reg = trial.suggest_loguniform('l2_reg', 1e-5, 1e-2)
    
    # Tasa de aprendizaje
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    
    # Crear el modelo
    model = Sequential([
        Dense(num_neurons_1, activation='elu', kernel_regularizer=l2(l2_reg), input_dim=X_train_scaled.shape[1]),
        Dense(num_neurons_2, activation='elu', kernel_regularizer=l2(l2_reg)),
        Dense(y_train_encoded.shape[1], activation='softmax')
    ])
    
    # Compilar el modelo
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

def objective(trial):
    # Crear el modelo con los hiperparámetros sugeridos
    model = create_model(trial)
    
    # Definir el EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Entrenar el modelo
    history = model.fit(
        X_train_scaled, y_train_encoded,
        epochs=200,
        batch_size=8,
        validation_data=(X_val_scaled, y_val_encoded),
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Obtener la exactitud en el conjunto de validación
    val_accuracy = history.history['val_accuracy'][-1]
    
    return val_accuracy  # Maximizar la exactitud en validación

# Crear el estudio de Optuna
study = optuna.create_study(direction='maximize')

# Ejecutar la optimización
study.optimize(objective, n_trials=50)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:")
print(study.best_params)

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de test
best_model = create_model(study.best_trial)
best_model.fit(
    X_train_scaled, y_train_encoded,
    epochs=200,
    batch_size=8,
    validation_data=(X_val_scaled, y_val_encoded),
    callbacks=[early_stopping],
    verbose=1
)

# Evaluar en test
test_loss, test_accuracy = best_model.evaluate(X_test_scaled, y_test_encoded, verbose=0)
print(f"Exactitud en test con los mejores hiperparámetros: {test_accuracy:.4f}")


ModuleNotFoundError: No module named 'optuna'